### This is a test for adding padding and new DFT lengths


In [1]:
import numpy as np
from scipy.signal import chirp, spectrogram
import simpleaudio as sa
from scipy.io.wavfile import write, read
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt
from transmitter import *
from reciever import *
import random

In [2]:
fs = 44100
class OFDM:
    pass
ofdm = OFDM()
ofdm.N = 2048                   # Length of DTF
ofdm.start_bin = 48
ofdm.end_bin = 468
ofdm.CP = 256                      
ofdm.sfp = 20                   # Symbols per frame
ofdm.pilot_tones = [0]          # Pilot locations
ofdm.num_tones = 0              # Number of pilot tones
ofdm.bps = 2                    # bits per symbol QFSK
ofdm.preamb = ofdm.N//2
ofdm.QFSK_dict = {              # The constellation
    (1,1) : -1-1j,
    (1,0) :  1-1j,
    (0,0) :  1+1j,
    (0,1) : -1+1j}

In [ ]:
        ### Data forming With image
#bit_array = bitsFromTiff('jaguar.tiff')

        ### Data from text
bit_array = 
print('Length of bits', len(bit_array))


payload = fullTrans(bit_array, ofdm)
#flat_payload = payload.flatten() / np.max(np.abs(payload.flatten()))
print("payload shape:",np.shape(payload))


        ### Pilot symbol for channel estimation
qfsk = np.array(list(ofdm.QFSK_dict.values()))
pilot_block = np.random.choice(qfsk, size=(int(ofdm.N/2-1)), replace=True)
pilot_symbs = goodSymbols(pilot_block, ofdm)
pilot_full = addGuard(inversedft(pilot_symbs), ofdm)
pilot_full = pilot_full.flatten() / np.max(np.abs(pilot_full.flatten()))


        ### Chirp makeing for sync
chirp_time = 5*(ofdm.N+ofdm.CP) / fs
t = np.linspace(0, chirp_time, int(chirp_time * fs), False)
note = chirp(t, f0=500, f1=15000, t1=chirp_time, method='linear')
chirp_symbol = note / np.max(np.abs(note))

In [ ]:
plt.plot(payload.flatten()[50000:51000])
x = deconstruct(payload.flatten(), ofdm, numpy_func = True, channel_H = None, retSymbs = False)
bitsToSaveImage(x[:214400], 'trialpayload', 67, 100)

In [ ]:
emtpy = np.zeros(ofdm.N)

frame = frameMaker(chirp_symbol, pilot_full, payload, ofdm.ofdmSymbolsPerFrame, zeros_post_sync=emtpy, zeros_post_sybm=emtpy)
print("Frame Shape:",np.shape(frame))
audio_for_file = audioMaker(frame, 'multiframetest1', fs)

print("audio file length (samples, seconds):",len(audio_for_file), len(audio_for_file)*1/fs)

In [ ]:
plt.figure(figsize = (30, 10))
plt.plot(audio_for_file[7500:12000])
print('Length of audio:',len(audio_for_file))
print('Expected length of audio:', (len(payload[0:5].flatten())+len(chirp_symbol)+len(emtpy)*2+len(pilot_full)))
print('Expected length of audio:', len(payload.flatten())+len(chirp_symbol)*42+len(emtpy)*42*2)

In [ ]:
record(18,fs,'big_frame_image') 

In [ ]:
Fs, aud = read('big_frame_image.wav')

powerSpectrum, frequenciesFound, time, imageAxis = plt.specgram(aud, Fs=fs)
plt.show()

In [ ]:
plt.plot(np.correlate(aud, chirp_symbol, mode='full')[109200:109400])